In [0]:
dbutils.widgets.text("p_environment", "")

In [0]:
v_environment = dbutils.widgets.get("p_environment")
print(v_environment)

Paso 1- Leer el aerchivo CSV usando "DataFramereader" de spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

In [0]:
language_schema = StructType( fields = [
    StructField("languageId", IntegerType(), False),
    StructField("languageCode", StringType(), True),
    StructField("languageName", StringType(), True)
] )
                          

In [0]:
language_df = spark.read \
            .option("header", True) \
            .schema(language_schema) \
            .csv("dbfs:/mnt/moviehistoryl/bronce/language.csv")

In [0]:
%fs
ls /mnt/moviehistoryl/bronce

In [0]:
language_df.printSchema()

In [0]:
display(language_df.describe())

### Selecciona solo las columnas requeridas

In [0]:
# movies_selected_df = movie_df.select("movieId","title", "budget","popularity", "yearReleaseDate","ReleaseDate", "revenue","durationTime","voteAverage","voteCount")

In [0]:
# movies_selected_df = movie_df.select(movie_df["movieId"], movie_df["title"], movie_df["budget"],movie_df["popularity"], movie_df["yearReleaseDate"],movie_df["ReleaseDate"], movie_df["revenue"],movie_df["durationTime"], movie_df["voteAverage"],movie_df["voteCount"])

In [0]:
display(language_df)

In [0]:
from pyspark.sql.functions import col

### Cambiar nombre columnas segun lo requerido

In [0]:
language_selected_df = language_df.select(
    col("languageId").alias("language_id"), 
    col("languageName").alias("language_name")
)

In [0]:
display(language_selected_df)

### Agregar la columna "ingestion_date" al Dataframe

In [0]:
from pyspark.sql.functions import current_timestamp, lit


In [0]:
languages_final_df = language_selected_df.withColumn("ingestion_date", current_timestamp()) \
                                .withColumn("environment", lit(v_environment))

### Escriber datos en el data lake en formato parquet

In [0]:
languages_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.languages")

In [0]:
%fs
ls /mnt/moviehistoryl/silver/languages

In [0]:
df = spark.read.parquet("/mnt/moviehistoryl/silver/languages")

In [0]:
display(df)

In [0]:
dbutils.notebook.exit("success")